In [ ]:
%pip install --upgrade ipykernel

In [ ]:
import math
import numpy as np
import pandas as pd
import random

In [ ]:

def read_tsplib(filename):
    states_coords = {}
    with open(filename, 'r') as f:
        lines = [l.strip() for l in f.readlines()]
    section = False
    for line in lines:
        if line.upper().startswith('NODE_COORD_SECTION'):
            section = True
            continue
        if line.upper().startswith('EOF'):
            break
        if section:
            parts = line.split()
            if len(parts) >= 3:
                i = int(parts[0])
                x, y = float(parts[1]), float(parts[2])
                states_coords[i] = (x, y)
    states_coords = [states_coords[k] for k in sorted(states_coords.keys())]
    return states_coords

In [ ]:
def distance_matrix(states_coords): 
    n = len(states_coords)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                x1, y1 = states_coords[i]
                x2, y2 = states_coords[j]
                distance_matrix[i, j] = math.hypot(x1 - x2, y1 - y2)
    return distance_matrix
                
def reward_matrix(distance_matrix): 
     return -distance_matrix


In [ ]:
#D_df = pd.DataFrame(distance_matrix)
#D_df.head()

In [ ]:
#R_df = pd.DataFrame(reward_matrix)
#R_df.head()

In [ ]:

def epsilon_greedy(Q, s, actions, epsilon):
    
    n_a = random.random()

    if n_a < epsilon:
        # Ação aleatória (a_a)
        a = random.choice(actions)
    else:
        # Aação gulosa (a*)
        q_vals = [Q[s, act] for act in actions]
        best_index = int(np.argmax(q_vals))  
        a = actions[best_index]

    return a

In [ ]:
def q_learning(reward_matrix, alpha=0.1, gamma=0.9, epsilon=0.2, episodes=1000):
    n = R.shape[0]
    Q = np.zeros_like(R)
    
    for ep in range(episodes):
            # inicia em um estado aleatório
            s = random.randint(0, n - 1)

            while True:
                
                # Escolha a ação a (e-greedy)
                possible_actions = [a for a in range(n) if a != s]
                if not possible_actions:
                    break
                a = epsilon_greedy(Q, s, possible_actions, epsilon)

                # Receba a recompensa imediata r(s,a)
                r = reward_matrix[s, a]

                # Observe o novo estado s'
                s_next = a

                # Atualize o item Q(s,a)
                Q[s, a] = Q[s, a] + alpha * (r + gamma * np.max(Q[s_next, :]) - Q[s, a])

                # Atualize o estado atual
                s = s_next

                # Critério de parada
                if random.random() < 0.05:  # evitar loop infinito
                    break
    return Q

In [ ]:
def calculate_length(route, D):
    total = 0
    for i in range(len(route) - 1):
        total += D[route[i], route[i + 1]]
    return total

def best_route(Q, start=0):
    n = Q.shape[0]
    route = [start]
    current = start
    visited = {start}

    for _ in range(n - 1):
        # ações possíveis: cidades ainda não visitadas
        possible_actions = [a for a in range(n) if a not in visited]
        if not possible_actions:
            break

        # escolhe a ação com maior valor Q
        next_city = possible_actions[np.argmax([Q[current, a] for a in possible_actions])]
        route.append(next_city)
        visited.add(next_city)
        current = next_city

    # fecha o ciclo (volta à cidade inicial)
    route.append(start)
    return route

route = best_route(Q, start=0)
total_distance = calculate_length(route, D)

print("Rota aprendida:", route)
print(f"Distância total: {total_distance:.2f}")


In [ ]:
import numpy as np
import random
import time  # para medir tempo

def q_learning_with_tracking(R, D, alpha=0.1, gamma=0.9, epsilon=0.2, episodes=1000):
    n = R.shape[0]
    Q = np.zeros_like(R)
    
    # listas e variáveis de monitoramento
    distances = []         
    best_distance = float('inf')
    best_episode = 0
    best_route_found = None
    
    # inicia contagem de tempo
    start_time = time.time()

    for ep in range(episodes):
        s = random.randint(0, n - 1)
        
        while True:
            actions = [a for a in range(n) if a != s]
            if not actions:
                break

            # política ε-greedy
            if random.random() < epsilon:
                a = random.choice(actions)
            else:
                a = actions[int(np.argmax([Q[s, act] for act in actions]))]

            r = R[s, a]
            s_next = a

            # atualização Q-learning
            Q[s, a] = Q[s, a] + alpha * (r + gamma * np.max(Q[s_next, :]) - Q[s, a])
            s = s_next

            if random.random() < 0.05:
                break
        
        # avalia rota aprendida no episódio
        route = best_route(Q, start=0)
        total_distance = calculate_length(route, D)
        distances.append(total_distance)

        # verifica se é a melhor até agora
        if total_distance < best_distance:
            best_distance = total_distance
            best_episode = ep + 1
            best_route_found = route
    
    # fim do cronômetro
    end_time = time.time()
    total_time = end_time - start_time
    avg_time_per_episode = total_time / episodes

    # média das distâncias
    avg_distance = np.mean(distances)

    return {
        "Q": Q,
        "distances": distances,
        "avg_distance": avg_distance,
        "best_distance": best_distance,
        "best_episode": best_episode,
        "best_route": best_route_found,
        "total_time": total_time,
        "avg_time_per_episode": avg_time_per_episode
    }


In [ ]:
Q, distances, avg_distance, best_distance, best_episode, best_route_found = q_learning_with_tracking(
    R, D, alpha=0.1, gamma=0.9, epsilon=0.2, episodes=2000
)

print(f"Média das distâncias: {avg_distance:.2f}")
print(f"Melhor distância encontrada: {best_distance:.2f}")
print(f"Episódio da melhor rota: {best_episode}")
print("Melhor rota aprendida:", best_route_found)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(distances)
plt.xlabel('Episódio')
plt.ylabel('Distância total da rota')
plt.title('Evolução da distância ao longo do treinamento')
plt.show()


In [ ]:
#"\\wsl.localhost\Ubuntu-22.04\home\elisaveloso\aprendizado_por_reforco\berlin52.tsp\berlin52.tsp"
coords = read_tsplib("/home/elisaveloso/aprendizado_por_reforco/berlin52.tsp/berlin52.tsp")
coords_small = coords[:52]   # usa só 10 cidades para teste
D = distance_matrix(coords_small)
R = -D                    

Q = q_learning(R, alpha=0.75, gamma=0.15, epsilon=0.01, episodes=1000)
print("Q-table aprendida:")
print(Q.round(2))
